# 3D Information Processing
Obtain a 3D shape from images taken by multiple cameras

In [1]:
import pandas as pd
from camera import Camera
from stereo import Stereo
from visualize import plot_calibration_points

Load calibration points for each images

In [2]:
points_1 = pd.read_csv("points/points_1.csv")
points_2 = pd.read_csv("points/points_2.csv")
points_3 = pd.read_csv("points/points_3.csv")

Plot calibration points

In [3]:
plot_calibration_points("data/1.JPG", "data/1_plotted.JPG", points_1)
plot_calibration_points("data/2.JPG", "data/2_plotted.JPG", points_2)
plot_calibration_points("data/3.JPG", "data/3_plotted.JPG", points_3)

### 1. Obtain perspective projection matrix

Calibrate cameras from calibration points

In [4]:
c1 = Camera(points_1)
c2 = Camera(points_2)
c3 = Camera(points_3)

In [5]:
c1.calibrate()
c2.calibrate()
c3.calibrate()

Calibrate camera...
Perspective Projection Matrix
[[-4.05412297e+01  4.03725932e+01 -2.20953401e+01  1.57180272e+03]
 [ 1.46741972e+01  5.29694456e+00 -5.53211850e+01  1.11837795e+03]
 [-1.17373954e-02 -4.11469771e-03 -1.11584529e-02  1.00000000e+00]]
Calibrate camera...
Perspective Projection Matrix
[[-4.75848755e+01  3.16342338e+01 -1.80047358e+01  1.58283552e+03]
 [ 6.04330571e+00  2.71247349e+00 -5.57989173e+01  1.08335787e+03]
 [-1.20918587e-02 -6.46704780e-03 -8.86434477e-03  1.00000000e+00]]
Calibrate camera...
Perspective Projection Matrix
[[-4.68468219e+01  2.77961801e+01 -2.39493487e+01  1.67369448e+03]
 [ 1.67609536e+01  1.20751963e+01 -5.18018979e+01  8.89834552e+02]
 [-8.88890611e-03 -6.40756403e-03 -1.20972384e-02  1.00000000e+00]]


Validate perspective projection matrix by other points

In [6]:
points_valid = pd.read_csv("points/points_1_valid.csv")
pred_u = []
pred_v = []
for row in points_valid.itertuples():
    u_, v_ = c1.perspective_project(row[3], row[4], row[5])
    pred_u += [u_]
    pred_v += [v_]

predict = pd.DataFrame({"u": pred_u, "v": pred_v,
                        "x": points_valid["x"],
                        "y": points_valid["y"],
                        "z": points_valid["z"]})

predict["u"] = predict["u"].astype(int)
predict["v"] = predict["v"].astype(int)

plot_calibration_points("data/1.JPG", "data/1_pred.JPG", points_valid)
plot_calibration_points("data/1_pred.JPG", "data/1_pred.JPG",
                        predict, color=(0, 0, 230))

Re-project calibration points

In [7]:
c1.re_project()

plot_calibration_points("data/1.JPG", "data/1_reproject.JPG",
                        c1.points_for_calibration)
plot_calibration_points("data/1_reproject.JPG", "data/1_reproject.JPG",
                        c1.points_of_reprojection,
                        color=(0, 0, 230))
reprojection_error = c1.re_projection_error()
reprojection_error

6.332554156756919

In [8]:
c2.re_project()

plot_calibration_points("data/2.JPG", "data/2_reproject.JPG",
                        c2.points_for_calibration)
plot_calibration_points("data/2_reproject.JPG", "data/2_reproject.JPG",
                        c2.points_of_reprojection,
                        color=(0, 0, 230))

reprojection_error = c2.re_projection_error()
reprojection_error

5.146138217502861

In [9]:
c3.re_project()

plot_calibration_points("data/3.JPG", "data/3_reproject.JPG",
                        c3.points_for_calibration)
plot_calibration_points("data/3_reproject.JPG", "data/3_reproject.JPG",
                        c3.points_of_reprojection,
                        color=(0, 0, 230))

reprojection_error = c3.re_projection_error()
reprojection_error

5.419570702840029

### 2. Obtain 3D points from 2 images

Obtain 3D points from 2 images

In [10]:
points12 = pd.read_csv("points/points_1_2.csv")
s = Stereo(c1, c2, points12)
s.obtain_objects_points_by_stereo()

(X, Y, Z) =  [25.46853018 16.45891675  0.27088423]
(X, Y, Z) =  [24.74322514 17.17460602  0.21459361]
(X, Y, Z) =  [21.9623529  20.02167334  0.32099857]
(X, Y, Z) =  [19.91808882 22.15987022  0.38459758]
(X, Y, Z) =  [16.4237593  25.69960559  0.35387484]
(X, Y, Z) =  [5.25763004 7.23655657 9.16204878]
(X, Y, Z) =  [10.59512338  5.50404891  9.27484466]
(X, Y, Z) =  [9.37281706 8.25184944 9.13307199]
(X, Y, Z) =  [ 8.15256931 11.08960475  9.13429177]
(X, Y, Z) =  [4.45047282 9.9362386  9.156412  ]
(X, Y, Z) =  [11.59571684  6.51458292  2.8439244 ]
(X, Y, Z) =  [10.64963668 10.55205759  6.59275462]
(X, Y, Z) =  [11.65461478  6.63672553  6.33230488]
(X, Y, Z) =  [ 9.52752728 11.54604342  8.79831639]
(X, Y, Z) =  [ 9.39823315 11.52663161  2.80247845]
(X, Y, Z) =  [13.09258375 17.84186136  6.14589095]
(X, Y, Z) =  [15.30964117 17.3783268   1.98313607]
(X, Y, Z) =  [13.30651068 21.42861199  2.55998543]
(X, Y, Z) =  [18.57716264 12.74336519  6.16123568]
(X, Y, Z) =  [ 8.38610995 27.03306098  9

In [11]:
s.points_of_objects

,X,Y,Z
0,25.468530,16.458917,0.270884
1,24.743225,17.174606,0.214594
2,21.962353,20.021673,0.320999
3,19.918089,22.159870,0.384598
4,16.423759,25.699606,0.353875
5,5.257630,7.236557,9.162049
6,10.595123,5.504049,9.274845
7,9.372817,8.251849,9.133072
8,8.152569,11.089605,9.134292
9,4.450473,9.936239,9.156412


Re-project from obatined 3D points

In [12]:
points_1_ = pd.DataFrame({"u": points12["u1"], "v": points12["v1"],
                          "x": s.points_of_objects["X"],
                          "y": s.points_of_objects["Y"],
                          "z": s.points_of_objects["Z"]})
points_2_ = pd.DataFrame({"u": points12["u2"], "v": points12["v2"],
                          "x": s.points_of_objects["X"],
                          "y": s.points_of_objects["Y"],
                          "z": s.points_of_objects["Z"]})

In [13]:
re_points_1_ = c1.perspective_project_points(points_1_)
re_points_2_ = c2.perspective_project_points(points_2_)
re_points_1_

,u,v,x,y,z
0,1900,2481,25.468530,16.458917,0.270884
1,1975,2451,24.743225,17.174606,0.214594
2,2259,2329,21.962353,20.021673,0.320999
3,2460,2246,19.918089,22.159870,0.384598
4,2775,2115,16.423759,25.699606,0.353875
5,1796,901,5.257630,7.236557,9.162049
6,1547,1053,10.595123,5.504049,9.274845
7,1754,1053,9.372817,8.251849,9.133072
8,1965,1045,8.152569,11.089605,9.134292
9,1976,906,4.450473,9.936239,9.156412


In [14]:
plot_calibration_points("data/1.JPG", "data/1_stereo_reproject.JPG", points_1_)
plot_calibration_points("data/1_stereo_reproject.JPG", "data/1_stereo_reproject.JPG",
                        re_points_1_, color=(0, 0, 230))

plot_calibration_points("data/2.JPG", "data/2_stereo_reproject.JPG", points_2_)
plot_calibration_points("data/2_stereo_reproject.JPG", "data/2_stereo_reproject.JPG",
                        re_points_2_, color=(0, 0, 230))
